In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.data import Dataset
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Activation, GRU
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from tensorflow.keras.callbacks import LambdaCallback
from sklearn import tree
from sklearn.metrics import confusion_matrix
import itertools
import time
import multiprocessing

%matplotlib inline

gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
print(gpus)
tf.config.experimental.set_visible_devices(devices=gpus[0], device_type='GPU')
tf.config.experimental.set_memory_growth(device=gpus[0], enable=True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())
# gpus = tf.config.experimental.list_physical_devices(device_type='CPU')
# print(gpus)

In [3]:
pwd = os.getcwd()
all_file_names = []
scenarios = ['scenario'+str(i+1) for i in range(5)]
data_dist_table = pd.DataFrame(index=['Hold','Takeoff','Hover','Search','Loiter','Obstacleavoid','Land'])
for scenario in scenarios:
    count_dict = {'Hold':0,'Takeoff':0,'Hover':0,'Search':0,'Loiter':0,'Obstacleavoid':0,'Land':0}
    database_path = pwd[:-6] + "flightData/filtered_data/"+scenario
    database_file_names = []
    for root, dirs, files in os.walk(database_path):
        database_file_names = [os.path.join(root,file) for file in files]
    all_file_names += database_file_names
    for file in database_file_names:
        data = pd.read_csv(file)
        label_count = data['label'].value_counts()
        for indx in label_count.index:
            count_dict[indx] += label_count[indx]
    data_dist_table[scenario] = count_dict.values()
print(data_dist_table)            

               scenario1  scenario2  scenario3  scenario4  scenario5
Hold               13264      13287      64310      57328      23519
Takeoff             1813       1844       1900       1858       1801
Hover               9001       9000       9045       9001       9014
Search                 0     109788     204655     541196     319726
Loiter                 0          0      83451          0      51460
Obstacleavoid          0          0          0      17441      11533
Land                1940       2088       1986       1848       1896


In [6]:
# plt.figure(figsize=(18,16))
# # for i,file in enumerate(database_file_names):
# file = database_file_names[0]
# data = pd.read_csv(file)
# # i=1
# col = data.columns[0]
# # for col in data.columns:
# #     if col=='time':
# #         continue
# x = data['time'].values
# for i,col in enumerate(data.columns):
#     y = data[col].values
#     plt.subplot(7,2,i+1)
#     plt.plot(x,y,label=col)
#     plt.grid()
#     plt.legend()
# #     i=i+1
# #         data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
# #         data.to_csv(file,index=False)
# #     old_label = pd.unique(data['label'])
# #     print(old_label)
# #     new_label = [j for j in range(1,len(old_label)+1)]
# #     data = data.replace(old_label, new_label)
# #     #     print(file)
# #     x = data['x'].values
# #     y = data['y'].values
# #     t = data['time'].values
# #     label = data['label'].values
# # #     label[22800:25350] = 4
# #     label = label*10
# #     plt.subplot(25,2,i+1)
# #     #     plt.plot(t,x)
# #     indx = 0
# #     plt.plot(y[indx:])
# #     plt.plot(label[indx:])
# #     plt.grid()
# #     file_name = file.split("/")
# #     file_name = file_name[-1]
# #     plt.title(file_name)

# # newfile = file[:-4]+'_new.csv'
# # data = data.replace(new_label, old_label)
# # data.to_csv(newfile, index=False)

# #     print(i)

In [35]:
# data.head()
# print(x[1]-x[0])
# print(x[2]-x[1])
# print(x[3]-x[1])

In [6]:
# print([i for i in range(1,len(pd.unique(data['label']))+1)])

In [7]:
# print(pd.unique(data['label']))

In [8]:
# print(file[:-4]+'_new.csv')

In [9]:
# X_train = data.drop(columns=['label', 'time'])
# X_train = X_train.values
# print(X_train.shape)
# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_train)
# Y_train = data['label'].values
# Y_train = Y_train.reshape((-1,1))
# encoder = OneHotEncoder()
# Y_train = encoder.fit_transform(Y_train).toarray()
# print(Y_train.shape)
# print(Y_train[0,:])

In [21]:
def batch_gen(X,y,time_step,batch_size=64):
    i = 0
    m = X.shape[0]
    print("Total train samples {}".format(m))
    
    while True:
        x_batch = []
        y_batch = []
        
        for j in range(batch_size):
            if i+j+time_step == m:
                i = -1
                break
            x_batch.append(X[i+j:i+j+time_step,:])
            y_batch.append(y[i+j+time_step,:])
        
        yield (np.array(x_batch), np.array(y_batch), i)
        i = i+1
        
        
def batch_gen_MLP(X,y,time_step,batch_size=64):
    m = X.shape[0]
    n = y.shape[1]
    i = 0
    while True:
        x_batch = []
        y_batch = [] 
        for j in range(batch_size):
            if i+j+time_step == m:
                i = -1
                break
            x_batch.append((X[i+j:i+j+time_step,:]).reshape((1,-1)))
            arg = np.argmax(np.sum(y[i+j:i+j+time_step,:], axis=0))
            label = np.zeros((1,n))
            label[0,arg] = 1.0
            y_batch.append(label)
        if i != -1:
            x_batch = (np.array(x_batch)).reshape((batch_size,-1))
            y_batch = (np.array(y_batch)).reshape((batch_size,-1))
        else:
            x_batch = (np.array(x_batch)).reshape((j,-1))
            y_batch = (np.array(y_batch)).reshape((j,-1))
        
        yield (x_batch, y_batch, i)
        i = i+1

def batch_gen_mlp(X,y,batch_size=64):
    m = X.shape[0]
    flag = 0
    i = 0
    while True:
        yield (X[i*batch_size:(i+1)*batch_size,:], y[i*batch_size:(i+1)*batch_size,:], (-1 if ((i+1)*batch_size > m) else 0))
        i += 1
        if i*batch_size > m:
            i = 0

In [5]:
# scaler = MinMaxScaler()
# encoder = OneHotEncoder()
# error = []
# file = database_file_names[0]
# data = pd.read_csv(file)
# X_train = data.drop(columns=['label', 'time','x','y','pitch_v','roll_v','yaw_v'])
# X_train = X_train.values
# Y_train = data['label'].values
# Y_train = Y_train.reshape((-1,1))

# X_train = scaler.fit_transform(X_train)
# Y_train = encoder.fit_transform(Y_train).toarray()

# #data_gen = batch_gen(X_train, Y_train, 32, 512)

# dataset = Dataset.from_generator(batch_gen, (tf.dtypes.variant,tf.dtypes.variant,tf.dtypes.int64))

# strategy = tf.distribute.MirroredStrategy()
# print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [4]:
# # deep LSTM model
# time_step = 64
# num_feature = 7
# num_class = 7
# bi = '1'
# neuron_size = 128
# lstm_layer = 1


# # x_in = Input(shape=(time_step, num_feature),dtype="float32")
# # x, _, _ = LSTM(units=64, return_sequences=True, return_state=True, activation='tanh')(x_in)
# # x, _, _ = LSTM(units=128, return_sequences=True, return_state=True, activation='tanh')(x)
# # _, _, x = LSTM(units=64, return_sequences=True, return_state=True, activation='tanh')(x)
# # x = Dense(units=128, activation="softmax")(x)
# # x = Dense(units=256, activation="softmax")(x)
# # y = Dense(units=num_class, activation="softmax")(x)

# # whole_sequence_output, final_state
# # x_in = Input(shape=(num_feature),dtype="float32")
# # x = Dense(units=64, activation="relu")(x_in)
# # x = Dense(units=128, activation="relu")(x)
# # x = Dense(units=64, activation="relu")(x)
# # y = Dense(units=num_class, activation="softmax")(x)

# x_in = Input(shape=(time_step,num_feature),dtype="float32")
# # forward_layer = LSTM(units=128, return_sequences=True, activation='tanh')
# # backward_layer = LSTM(units=128, return_sequences=True, activation='tanh', go_backwards=True)
# # bid_layer1 = Bidirectional(layer=forward_layer,backward_layer=backward_layer, merge_mode='concat')(x_in)
# bid_layer2 = Bidirectional(LSTM(neuron_size))(x_in)
# # x = Dense(units=128, activation="relu")(bid_layer[:,-1,:])
# # x = Dense(units=256, activation="relu")(x)
# y = Dense(units=num_class, activation='softmax')(bid_layer2)

# model = Model(inputs=x_in, outputs=y)
# optimizer = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
# # loss_fn = losses.CategoricalCrossentropy(from_logits=True)
# # train_acc_metric = metrics.CategoricalAccuracy()
# loss = losses.categorical_crossentropy
# model.compile(optimizer=optimizer, loss=loss, metrics=['CategoricalAccuracy'])
# model.summary()

In [16]:
# #create LSTM model
# num_feature = 7
# time_step = 32
# num_class = 7
# x_in = Input(shape=(time_step,num_feature),dtype="float32")
# forward_layer = LSTM(units=128, return_sequences=True, activation='tanh')
# backward_layer = LSTM(units=128, return_sequences=True, activation='tanh', go_backwards=True)
# bid_layer = Bidirectional(layer=forward_layer,backward_layer=backward_layer, merge_mode='concat')(x_in)
# y = Dense(units=num_class,activation='softmax')(bid_layer[:,-1,:])
# model = Model(inputs=x_in,outputs=y)
# optimizer = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
# loss_fn = losses.CategoricalCrossentropy(from_logits=True)
# train_acc_metric = metrics.CategoricalAccuracy()
# #loss = losses.categorical_crossentropy
# #model.compile(optimizer=optimizer, loss=loss, metrics=['CategoricalAccuracy'])


In [23]:
# class Process(multiprocessing.Process): 
#     def __init__(self, id, model,loss_fn, x_batch_train, y_batch_train): 
#         super(Process, self).__init__() 
#         self.id = id
#         self.grads = None
#         self.model = model
#         self.loss_fn = loss_fn
#         self.x_batch_train = x_batch_train
#         self.y_batch_train = y_batch_train
                 
#     def run(self): 
#         with tf.GradientTape() as tape:
#                 logits = self.model(self.x_batch_train, training=True)
#                 loss_value = self.loss_fn(self.y_batch_train, logits)
#         self.grads = tape.gradient(loss_value, self.model.trainable_weights)

In [7]:
scaler = MinMaxScaler()
encoder = OneHotEncoder()
pwd = os.getcwd()
database_path = pwd[:-6] + "flightData/filtered_data/scenario5"
database_file_names = []
for root, dirs, files in os.walk(database_path):
    database_file_names = [os.path.join(root,file) for file in files]

data = pd.read_csv(database_file_names[0])
X_train = data.drop(columns=['label', 'time','x','y','pitch_v','roll_v','yaw_v'])
X_train = X_train.values
Y_train = data['label'].values
Y_train = Y_train.reshape((-1,1))
_ = scaler.fit_transform(X_train)
_ = encoder.fit_transform(Y_train).toarray()

In [ ]:
%%time

# training

# deep LSTM model
time_steps = [128] #[32, 64, 128]
num_features = [7,9]
num_class = 7
direction = 2
neuron_sizes = [64, 128, 256, 512]
lstm_layers = 1

for time_step in time_steps:
    for num_feature in num_features:
        for neuron_size in neuron_sizes:
            # create model
            x_in = Input(shape=(time_step,num_feature),dtype="float32")
            bid_layer1 = Bidirectional(LSTM(neuron_size))(x_in)
            y = Dense(units=num_class, activation='softmax')(bid_layer1)

            model = Model(inputs=x_in, outputs=y)
            optimizer = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
            loss = losses.categorical_crossentropy
            model.compile(optimizer=optimizer, loss=loss, metrics=['CategoricalAccuracy'])


            # load dataset
            pwd = os.getcwd()
            model_path = pwd+'/models/'

            batch_print_callback = LambdaCallback(on_batch_end=lambda batch,logs: print(batch, logs))

            pwd = os.getcwd()
            database_path = pwd[:-6] + "flightData/filtered_data/scenario5"
            database_file_names = []
            for root, dirs, files in os.walk(database_path):
                database_file_names = [os.path.join(root,file) for file in files]

            scaler = MinMaxScaler()
            encoder = OneHotEncoder()
            
            #training loop
            for model_no in range(1,71):
                for i,file in enumerate(database_file_names):
                    data = pd.read_csv(file)
                    if num_feature == 7:
                        X_train = data.drop(columns=['label', 'time','x','y','pitch_v','roll_v','yaw_v'])
                    else:
                        X_train = data.drop(columns=['label', 'time','pitch_v','roll_v','yaw_v'])

                    X_train = X_train.values
                    Y_train = data['label'].values
                    Y_train = Y_train.reshape((-1,1))
                    if i == 0:
                        X_train = scaler.fit_transform(X_train)
                        Y_train = encoder.fit_transform(Y_train).toarray()
                    else:
                        X_train = scaler.transform(X_train)
                        Y_train = encoder.transform(Y_train).toarray()

                    input_data = X_train[:-time_step,:]
                    targets = Y_train[time_step:,:]
                    dataset = timeseries_dataset_from_array(input_data, 
                                                            targets, 
                                                            sequence_length=time_step,
                                                            batch_size=128)
                    hist = model.fit(dataset,
                                     epochs=10,
                                     verbose=2,
                                     callbacks=None,#[batch_print_callback],
                                     max_queue_size=512,
                                     workers=80,
                                     use_multiprocessing=True)

                model_name = 'lstm_direction_' + str(direction)\
                             + '_layers_' + str(lstm_layers)\
                             + '_features_' + str(num_feature)\
                             + '_neurons_' + str(neuron_size)\
                             + '_timesteps_' + str(time_step)\
                             + '_epochs_' + str((model_no)*10) + '.h5'
                model.save(model_path+model_name) 


Epoch 1/10
602/602 - 19s - loss: 0.8201 - categorical_accuracy: 0.7336
Epoch 2/10
602/602 - 14s - loss: 0.7247 - categorical_accuracy: 0.7218
Epoch 3/10
602/602 - 14s - loss: 0.6662 - categorical_accuracy: 0.7767
Epoch 4/10
602/602 - 14s - loss: 0.5388 - categorical_accuracy: 0.8087
Epoch 5/10
602/602 - 14s - loss: 0.4717 - categorical_accuracy: 0.8656
Epoch 6/10
602/602 - 14s - loss: 0.5769 - categorical_accuracy: 0.8326
Epoch 7/10
602/602 - 14s - loss: 0.5256 - categorical_accuracy: 0.8238
Epoch 8/10
602/602 - 14s - loss: 0.4270 - categorical_accuracy: 0.8847
Epoch 9/10
602/602 - 14s - loss: 0.4159 - categorical_accuracy: 0.8656
Epoch 10/10
602/602 - 14s - loss: 0.3998 - categorical_accuracy: 0.8653
Epoch 1/10
859/859 - 20s - loss: 0.2363 - categorical_accuracy: 0.9211
Epoch 2/10
859/859 - 21s - loss: 0.2994 - categorical_accuracy: 0.9125
Epoch 3/10
859/859 - 20s - loss: 0.3296 - categorical_accuracy: 0.9096
Epoch 4/10
859/859 - 20s - loss: 0.2598 - categorical_accuracy: 0.9165
Epoch

Epoch 7/10
859/859 - 20s - loss: 0.3691 - categorical_accuracy: 0.8915
Epoch 8/10
859/859 - 20s - loss: 0.3736 - categorical_accuracy: 0.8908
Epoch 9/10
859/859 - 21s - loss: 0.3779 - categorical_accuracy: 0.8938
Epoch 10/10
859/859 - 21s - loss: 0.3726 - categorical_accuracy: 0.8930
Epoch 1/10
903/903 - 21s - loss: 0.3572 - categorical_accuracy: 0.8968
Epoch 2/10
903/903 - 21s - loss: 0.3874 - categorical_accuracy: 0.8794
Epoch 3/10
903/903 - 22s - loss: 0.4065 - categorical_accuracy: 0.8875
Epoch 4/10
903/903 - 22s - loss: 0.4176 - categorical_accuracy: 0.8936
Epoch 5/10
903/903 - 21s - loss: 0.3422 - categorical_accuracy: 0.8932
Epoch 6/10
903/903 - 22s - loss: 0.3605 - categorical_accuracy: 0.8914
Epoch 7/10
903/903 - 22s - loss: 0.3248 - categorical_accuracy: 0.8852
Epoch 8/10
903/903 - 21s - loss: 0.3288 - categorical_accuracy: 0.8795
Epoch 9/10
903/903 - 22s - loss: 0.3425 - categorical_accuracy: 0.8843
Epoch 10/10
903/903 - 21s - loss: 0.3432 - categorical_accuracy: 0.8868
Epoc

Epoch 3/10
466/466 - 11s - loss: 0.4840 - categorical_accuracy: 0.8366
Epoch 4/10
466/466 - 11s - loss: 0.4736 - categorical_accuracy: 0.8383
Epoch 5/10
466/466 - 11s - loss: 0.4703 - categorical_accuracy: 0.8372
Epoch 6/10
466/466 - 11s - loss: 0.5361 - categorical_accuracy: 0.8182
Epoch 7/10
466/466 - 11s - loss: 0.4945 - categorical_accuracy: 0.8340
Epoch 8/10
466/466 - 12s - loss: 0.5201 - categorical_accuracy: 0.8264
Epoch 9/10
466/466 - 11s - loss: 0.4489 - categorical_accuracy: 0.8461
Epoch 10/10
466/466 - 11s - loss: 0.4492 - categorical_accuracy: 0.8372
Epoch 1/10
434/434 - 11s - loss: 0.5864 - categorical_accuracy: 0.7855
Epoch 2/10
434/434 - 11s - loss: 0.5677 - categorical_accuracy: 0.7800
Epoch 3/10
434/434 - 11s - loss: 0.5636 - categorical_accuracy: 0.7752
Epoch 4/10
434/434 - 11s - loss: 0.5491 - categorical_accuracy: 0.7856
Epoch 5/10
434/434 - 11s - loss: 0.5342 - categorical_accuracy: 0.7872
Epoch 6/10
434/434 - 11s - loss: 0.5645 - categorical_accuracy: 0.7121
Epoch

Epoch 9/10
434/434 - 10s - loss: 0.5003 - categorical_accuracy: 0.8250
Epoch 10/10
434/434 - 10s - loss: 0.4920 - categorical_accuracy: 0.8247
Epoch 1/10
602/602 - 14s - loss: 0.3634 - categorical_accuracy: 0.8913
Epoch 2/10
602/602 - 14s - loss: 0.4652 - categorical_accuracy: 0.8652
Epoch 3/10
602/602 - 14s - loss: 0.4137 - categorical_accuracy: 0.8701
Epoch 4/10
602/602 - 12s - loss: 0.4007 - categorical_accuracy: 0.8770
Epoch 5/10
602/602 - 14s - loss: 0.4575 - categorical_accuracy: 0.8698
Epoch 6/10
602/602 - 12s - loss: 0.4978 - categorical_accuracy: 0.8364
Epoch 7/10
602/602 - 12s - loss: 0.4696 - categorical_accuracy: 0.8513
Epoch 8/10
602/602 - 13s - loss: 0.4537 - categorical_accuracy: 0.8754
Epoch 9/10
602/602 - 13s - loss: 0.3863 - categorical_accuracy: 0.8779
Epoch 10/10
602/602 - 14s - loss: 0.3982 - categorical_accuracy: 0.8761
Epoch 1/10
859/859 - 19s - loss: 0.2958 - categorical_accuracy: 0.9026
Epoch 2/10
859/859 - 19s - loss: 0.3284 - categorical_accuracy: 0.9066
Epoc

Epoch 5/10
859/859 - 21s - loss: 0.2439 - categorical_accuracy: 0.9213
Epoch 6/10
859/859 - 21s - loss: 0.2655 - categorical_accuracy: 0.9213
Epoch 7/10
859/859 - 21s - loss: 0.2670 - categorical_accuracy: 0.9157
Epoch 8/10
859/859 - 21s - loss: 0.2513 - categorical_accuracy: 0.9086
Epoch 9/10
859/859 - 21s - loss: 0.2847 - categorical_accuracy: 0.9202
Epoch 10/10
859/859 - 21s - loss: 0.2648 - categorical_accuracy: 0.9073
Epoch 1/10
903/903 - 22s - loss: 0.2554 - categorical_accuracy: 0.9144
Epoch 2/10
903/903 - 22s - loss: 0.2656 - categorical_accuracy: 0.9161
Epoch 3/10
903/903 - 22s - loss: 0.2710 - categorical_accuracy: 0.9121
Epoch 4/10
903/903 - 22s - loss: 0.2037 - categorical_accuracy: 0.9240
Epoch 5/10
903/903 - 22s - loss: 0.1921 - categorical_accuracy: 0.9328
Epoch 6/10
903/903 - 22s - loss: 0.2202 - categorical_accuracy: 0.9204
Epoch 7/10
903/903 - 22s - loss: 0.2130 - categorical_accuracy: 0.9238
Epoch 8/10
903/903 - 22s - loss: 0.2559 - categorical_accuracy: 0.9202
Epoch

...

Epoch 1/10
903/903 - 22s - loss: 1.3530 - categorical_accuracy: 0.8118
Epoch 2/10
903/903 - 22s - loss: 1.3573 - categorical_accuracy: 0.8442
Epoch 3/10
903/903 - 22s - loss: 0.9717 - categorical_accuracy: 0.8472
Epoch 4/10
903/903 - 22s - loss: 0.8216 - categorical_accuracy: 0.8602
Epoch 5/10
903/903 - 22s - loss: 0.7925 - categorical_accuracy: 0.8737
Epoch 6/10
903/903 - 22s - loss: 0.6317 - categorical_accuracy: 0.8833
Epoch 7/10
903/903 - 22s - loss: 0.5969 - categorical_accuracy: 0.8939
Epoch 8/10
903/903 - 22s - loss: 0.9838 - categorical_accuracy: 0.8580
Epoch 9/10
903/903 - 22s - loss: 0.9597 - categorical_accuracy: 0.8604
Epoch 10/10
903/903 - 22s - loss: 0.8539 - categorical_accuracy: 0.8653
Epoch 1/10
466/466 - 12s - loss: 1.4926 - categorical_accuracy: 0.7652
Epoch 2/10
466/466 - 11s - loss: 3.2108 - categorical_accuracy: 0.6798
Epoch 3/10
466/466 - 12s - loss: 1.6165 - categorical_accuracy: 0.6901
Epoch 4/10
466/466 - 11s - loss: 1.0553 - categorical_accuracy: 0.7781
Epoch

Epoch 7/10
466/466 - 11s - loss: 0.4364 - categorical_accuracy: 0.8394
Epoch 8/10
466/466 - 12s - loss: 0.5169 - categorical_accuracy: 0.8304
Epoch 9/10
466/466 - 11s - loss: 0.4377 - categorical_accuracy: 0.8458
Epoch 10/10
466/466 - 11s - loss: 0.4235 - categorical_accuracy: 0.8453
Epoch 1/10
434/434 - 11s - loss: 0.5118 - categorical_accuracy: 0.8379
Epoch 2/10
434/434 - 11s - loss: 0.5552 - categorical_accuracy: 0.8174
Epoch 3/10
434/434 - 11s - loss: 0.4591 - categorical_accuracy: 0.8404
Epoch 4/10
434/434 - 11s - loss: 0.5263 - categorical_accuracy: 0.8488
Epoch 5/10
434/434 - 11s - loss: 0.6373 - categorical_accuracy: 0.8253
Epoch 6/10
434/434 - 11s - loss: 0.4172 - categorical_accuracy: 0.8647
Epoch 7/10
434/434 - 11s - loss: 0.4020 - categorical_accuracy: 0.8542
Epoch 8/10
434/434 - 11s - loss: 0.4033 - categorical_accuracy: 0.8735
Epoch 9/10
434/434 - 11s - loss: 0.4971 - categorical_accuracy: 0.8483
Epoch 10/10
434/434 - 11s - loss: 0.6946 - categorical_accuracy: 0.8353
Epoc

Epoch 3/10
602/602 - 15s - loss: 0.3866 - categorical_accuracy: 0.8817
Epoch 4/10
602/602 - 15s - loss: 0.3360 - categorical_accuracy: 0.8987
Epoch 5/10
602/602 - 15s - loss: 0.3843 - categorical_accuracy: 0.8918
Epoch 6/10
602/602 - 15s - loss: 0.3493 - categorical_accuracy: 0.8908
Epoch 7/10
602/602 - 15s - loss: 0.3650 - categorical_accuracy: 0.8847
Epoch 8/10
602/602 - 15s - loss: 0.3505 - categorical_accuracy: 0.8855
Epoch 9/10
602/602 - 15s - loss: 0.3666 - categorical_accuracy: 0.8897
Epoch 10/10
602/602 - 15s - loss: 0.3255 - categorical_accuracy: 0.8933
Epoch 1/10
859/859 - 21s - loss: 0.2361 - categorical_accuracy: 0.9208
Epoch 2/10
859/859 - 21s - loss: 0.2320 - categorical_accuracy: 0.9276
Epoch 3/10
859/859 - 21s - loss: 0.3251 - categorical_accuracy: 0.9090
Epoch 4/10
859/859 - 21s - loss: 0.2706 - categorical_accuracy: 0.9196
Epoch 5/10
859/859 - 21s - loss: 0.2893 - categorical_accuracy: 0.9265
Epoch 6/10
859/859 - 21s - loss: 0.2357 - categorical_accuracy: 0.9318
Epoch

Epoch 9/10
859/859 - 21s - loss: 0.1879 - categorical_accuracy: 0.9375
Epoch 10/10
859/859 - 21s - loss: 0.2200 - categorical_accuracy: 0.9400
Epoch 1/10
903/903 - 22s - loss: 0.2363 - categorical_accuracy: 0.9345
Epoch 2/10
903/903 - 22s - loss: 0.1938 - categorical_accuracy: 0.9405
Epoch 3/10
903/903 - 22s - loss: 0.2969 - categorical_accuracy: 0.9234
Epoch 4/10
903/903 - 22s - loss: 0.2485 - categorical_accuracy: 0.9237
Epoch 5/10
903/903 - 22s - loss: 0.2550 - categorical_accuracy: 0.9306
Epoch 6/10
903/903 - 22s - loss: 0.2498 - categorical_accuracy: 0.9373
Epoch 7/10
903/903 - 22s - loss: 0.2124 - categorical_accuracy: 0.9351
Epoch 8/10
903/903 - 22s - loss: 0.2282 - categorical_accuracy: 0.9332
Epoch 9/10
903/903 - 22s - loss: 0.2244 - categorical_accuracy: 0.9361
Epoch 10/10
903/903 - 22s - loss: 0.1969 - categorical_accuracy: 0.9409
Epoch 1/10
466/466 - 11s - loss: 0.4468 - categorical_accuracy: 0.8721
Epoch 2/10
466/466 - 11s - loss: 0.3539 - categorical_accuracy: 0.8836
Epoc

Epoch 5/10
466/466 - 11s - loss: 0.4418 - categorical_accuracy: 0.8978
Epoch 6/10
466/466 - 12s - loss: 0.3257 - categorical_accuracy: 0.9160
Epoch 7/10
466/466 - 12s - loss: 0.3252 - categorical_accuracy: 0.9157
Epoch 8/10
466/466 - 11s - loss: 0.3178 - categorical_accuracy: 0.9178
Epoch 9/10
466/466 - 11s - loss: 0.3028 - categorical_accuracy: 0.9128
Epoch 10/10
466/466 - 12s - loss: 0.3055 - categorical_accuracy: 0.9181
Epoch 1/10
434/434 - 11s - loss: 0.5246 - categorical_accuracy: 0.8671
Epoch 2/10
434/434 - 11s - loss: 0.5185 - categorical_accuracy: 0.8642
Epoch 3/10
434/434 - 11s - loss: 0.4715 - categorical_accuracy: 0.8667
Epoch 4/10
434/434 - 11s - loss: 0.4458 - categorical_accuracy: 0.8700
Epoch 5/10
434/434 - 11s - loss: 0.4589 - categorical_accuracy: 0.8733
Epoch 6/10
434/434 - 11s - loss: 0.4245 - categorical_accuracy: 0.8751
Epoch 7/10
434/434 - 11s - loss: 0.4216 - categorical_accuracy: 0.8730
Epoch 8/10
434/434 - 11s - loss: 0.4647 - categorical_accuracy: 0.8676
Epoch

In [21]:
%%time

pwd = os.getcwd()
model_path = pwd+'/models/'

model_wise_acc = {}

for model_no in range(1,101):

    model_name = 'lstm_model_blstm_' + str(model_no*10) + '.h5'
    model = load_model(model_path+model_name)

    acc_by_scenario = {'scenario'+str(i+1):[[],[]] for i in range(5)}
    prev_key = None
    x_values = []
    y_values = []

    for i,file in enumerate(all_file_names):
        data = pd.read_csv(file)
        X_test = data.drop(columns=['label', 'time','x','y','pitch_v','roll_v','yaw_v'])
        X_test = X_test.values
        Y_test = data['label'].values
        Y_test = Y_test.reshape((-1,1))

        X_test = scaler.transform(X_test)
        Y_test = encoder.transform(Y_test).toarray()

        input_data = X_test[:-time_step,:]
        targets = Y_test[time_step:,:]
        dataset = timeseries_dataset_from_array(input_data, 
                                                targets, 
                                                sequence_length=time_step,
                                                batch_size=128)
        hist = model.evaluate(dataset,
                              verbose=1,
                              callbacks=None,#[batch_print_callback],
                              max_queue_size=512,
                              workers=80,
                              use_multiprocessing=True,
                              return_dict=True)

        key = file.split('/')[-2]
        acc_by_scenario[key][0].append(int(((file.split('/')[-1]).split('_')[-1]).split('.')[0]))
        acc_by_scenario[key][1].append(hist['categorical_accuracy'])
    
    model_wise_acc[model_no] = acc_by_scenario


502/502 [==============================] - 4s 9ms/step - loss: 0.7667 - categorical_accuracy: 0.7583


435/435 [==============================] - 4s 9ms/step - loss: 0.6470 - categorical_accuracy: 0.7937


218/218 [==============================] - 2s 9ms/step - loss: 1.6010 - categorical_accuracy: 0.6601


414/414 [==============================] - 4s 9ms/step - loss: 0.2291 - categorical_accuracy: 0.9150


40/40 [==============================] - 0s 10ms/step - loss: 0.2894 - categorical_accuracy: 0.9408


...

531/531 [==============================] - 5s 9ms/step - loss: 4.2481 - categorical_accuracy: 0.6990


452/452 [==============================] - 4s 9ms/step - loss: 0.4269 - categorical_accuracy: 0.9035


212/212 [==============================] - 2s 10ms/step - loss: 7.8643 - categorical_accuracy: 0.4367


367/367 [==============================] - 4s 10ms/step - loss: 0.4824 - categorical_accuracy: 0.8848


860/860 [==============================] - 8s 10ms/step - loss: 1.0526 - categorical_accuracy: 0.6819
CPU times: user 22h 49min 32s, sys: 4h 43min 46s, total: 1d 3h 33min 18s
Wall time: 4h 36min 42s


In [38]:
model_wise_acc
# for i in range(1,6):
#     plt.scatter(acc_by_scenario['scenario'+str(i)][0],acc_by_scenario['scenario'+str(i)][1], label=str(i))
# plt.legend()
acc_by_scenario = {'scenario'+str(i+1):[] for i in range(5)}
for model_no in range(1,101):
    for scenerio_no in range(1,6):
        acc_by_scenario['scenario'+str(scenerio_no)].append(np.mean(model_wise_acc[model_no]['scenario'+str(scenerio_no)][1]))
        
for i in range(1,6):
#     plt.plot(acc_by_scenario['scenario'+str(i)], label=str(i))
    arg = np.argmax(acc_by_scenario['scenario'+str(i)])
    print(i,arg,acc_by_scenario['scenario'+str(i)][arg])
# plt.legend()

1 35 0.9577341973781586
2 3 0.9194792747497559
3 26 0.9115551872686907
4 5 0.9458085430992974
5 50 0.8815346240997315


In [59]:
pwd = os.getcwd()
database_path = pwd[:-6] + "flightData/filtered_data/scenario5"
database_file_names = []
for root, dirs, files in os.walk(database_path):
    database_file_names = [os.path.join(root,file) for file in files]

model_path = pwd+'/models/'
model_name = 'lstm_model_blstm_' + str(50*10) + '.h5'
model = load_model(model_path+model_name)

for i,file in enumerate(database_file_names):
        data = pd.read_csv(file)
        X_test = data.drop(columns=['label', 'time','x','y','pitch_v','roll_v','yaw_v'])
        X_test = X_test.values
        Y_test = data['label'].values
        Y_test = Y_test.reshape((-1,1))

        X_test = scaler.transform(X_test)
        Y_test = encoder.transform(Y_test).toarray()

        input_data = X_test[:-time_step,:]
        targets = Y_test[time_step:,:]
        dataset = timeseries_dataset_from_array(input_data, 
                                                targets, 
                                                sequence_length=time_step,
                                                batch_size=128)
        print(file)
#         hist = model.evaluate(dataset,
#                               verbose=1,
#                               callbacks=None,#[batch_print_callback],
#                               max_queue_size=512,
#                               workers=80,
#                               use_multiprocessing=True,
#                               return_dict=True)
        
        y_pred = model.predict(dataset,
                              verbose=1,
                              callbacks=None,#[batch_print_callback],
                              max_queue_size=512,
                              workers=80,
                              use_multiprocessing=True)
        
        y_pred = encoder.inverse_transform(y_pred)
        y_true = encoder.inverse_transform(targets[0:y_pred.shape[0],:])
        print(accuracy_score(y_true,y_pred))
        print(classification_report(y_true,y_pred))
        print(confusion_matrix(y_true,y_pred))
        
#         if i==0:
#             break

/home/ariac/ros-intel-uav-rpeo/simulation_ws/src/pie/flightData/filtered_data/scenario5/scenario5_4.csv
467/467 [==============================] - 4s 9ms/step
0.737733166039314
               precision    recall  f1-score   support

         Hold       0.87      0.91      0.89      5432
        Hover       0.00      0.00      0.00      1801
         Land       0.08      0.58      0.14       402
       Loiter       0.53      0.45      0.49      9863
Obstacleavoid       0.42      0.49      0.45      2401
       Search       0.84      0.84      0.84     39510
      Takeoff       1.00      0.23      0.38       366

     accuracy                           0.74     59775
    macro avg       0.53      0.50      0.45     59775
 weighted avg       0.74      0.74      0.74     59775

[[ 4932     0   202   298     0     0     0]
 [    0     0     0   335     0  1466     0]
 [    3     0   235    60     0   104     0]
 [  165    75   367  4436   673  4147     0]
 [    7     0   520    20  1167   6

In [61]:
# MLP model
time_step = 32
num_feature = 7
num_class = 7
x_in = Input(shape=(num_feature*time_step),dtype="float32")
x = Dense(units=64, activation="relu")(x_in)
x = Dense(units=128, activation="relu")(x)
x = Dense(units=256, activation="relu")(x)
x = Dense(units=256, activation="relu")(x)
x = Dense(units=128, activation="relu")(x)
x = Dense(units=64, activation="relu")(x)
y = Dense(units=num_class, activation="softmax")(x)
model = Model(inputs=x_in, outputs=y)
optimizer = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
loss = losses.categorical_crossentropy
model.compile(optimizer=optimizer, loss=loss, metrics=['CategoricalAccuracy'])

In [ ]:
#     for epoch in range(10):
#         j = 0
#         for (x_batch_train,y_batch_train,flag) in batch_gen(X_train, Y_train, time_step, 512):
# #             Process(j,model,loss_fn,x_batch_train,y_batch_train).start()
# #             j += 1
#             with tf.GradientTape() as tape:
#                 logits = model(x_batch_train, training=True)
#                 loss_value = loss_fn(y_batch_train, logits)
#             grads = tape.gradient(loss_value, model.trainable_weights)
            
#             optimizer.apply_gradients(zip(grads, model.trainable_weights))

#             # Update training metric.
#             train_acc_metric.update_state(y_batch_train, logits)
#             if flag==-1:
#                 break

#         # Display metrics at the end of each epoch.
#         train_acc = train_acc_metric.result()
#         print("Training acc over epoch: %.4f" % (float(train_acc),))

#         # Reset training metrics at the end of each epoch
#         train_acc_metric.reset_states()
#     if i==0:
#         break

In [82]:
print(model.summary())

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 32, 7)]           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 256)           139264    
_________________________________________________________________
tf.__operators__.getitem_1 ( (None, 256)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 7)                 1799      
Total params: 141,063
Trainable params: 141,063
Non-trainable params: 0
_________________________________________________________________
None


In [63]:
# plot_model(model=model,to_file="model.png")

In [64]:
# history = model.fit(batch_gen(X_train,Y_train,time_step,batch_size=32), epochs=100, verbose=2, shuffle=False)

In [45]:
#train
# # time_step = 0
# # model = load_model('lstm_model.h5')
# scaler = MinMaxScaler()
# encoder = OneHotEncoder()
# error = []
# for i,file in enumerate(database_file_names):
#     data = pd.read_csv(file)
#     X_train = data.drop(columns=['label', 'time','x','y','pitch_v','roll_v','yaw_v'])
#     X_train = X_train.values
#     Y_train = data['label'].values
#     Y_train = Y_train.reshape((-1,1))
#     if i == 0:
#         X_train = scaler.fit_transform(X_train)
#         Y_train = encoder.fit_transform(Y_train).toarray()
#     else:
#         X_train = scaler.transform(X_train)
#         Y_train = encoder.transform(Y_train).toarray()
    
# #     print(X_train.shape,Y_train.shape)
         
#     epoch = 0
#     er = []
#     #j = 0
#     hist = None
#     t1 = time.time()
#     for (x,y,flag) in batch_gen(X_train, Y_train, time_step, 512):
#         #j = j+1
#         #model.fit(x, y, epochs=1, verbose=2, shuffle=False)
# #         print(flag)
# #         print(x.shape,y.shape)
#         if flag == -1:
#             epoch += 1
#             print("epoch complete {}".format(epoch), hist)
#             print("This epoch took {} seconds".format(time.time()-t1))
#             error.append(np.mean(er))
#             er = []
#             model.save('lstm_model_f7.h5') 
#             t1 = time.time()
#         #if j%10000==0:
#         #    print(hist)
#         if epoch>10:
#             break
#         hist = model.train_on_batch(x,y)
#         er.append(hist[0])
# #         print(hist)
            
# plt.plot(error)
# plt.show()

In [45]:
# i=0
# for (x,y) in batch_gen(X_train,Y_train,time_step,32):
#     i=i+1
#     #model.fit(x, y, epochs=1, verbose=2, shuffle=False)
#     hist = model.test_on_batch(x,y)
#     if i%100==0:
#         print(hist)
#     if i>1000:
#         break
# print(error)
# model.save('lstm_model.h5')

In [80]:
file = database_file_names[1]
test_data = pd.read_csv(file)
X_test = test_data.drop(columns=['label', 'time','x','y','pitch_v','roll_v','yaw_v'])
X_test = X_test.values

X_test = scaler.transform(X_test)
Y_test = test_data['label'].values
Y_test = Y_test.reshape((-1,1))

Y_test = encoder.transform(Y_test).toarray()

i=0
for (x,y_true,flag) in batch_gen_MLP(X_test,Y_test,time_step,Y_test.shape[0]):
    y = model.predict(x)
    y_pred = np.zeros(y.shape)
    y = np.argmax(y,axis=1)
    y_pred[np.arange(y_pred.shape[0]),y] = 1
    y_pred = encoder.inverse_transform(y_pred)
    y_true = encoder.inverse_transform(y_true)
    
    i=i+1
    if i==1:
#         print(y_true)
        print(accuracy_score(y_true,y_pred))
        print(classification_report(y_true,y_pred))
        print(confusion_matrix(y_true,y_pred))
        break
    
# Y_test = encoder.transform(Y_test).toarray()

0.09233338900952062


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


               precision    recall  f1-score   support

         Hold       0.09      1.00      0.17      5528
        Hover       0.00      0.00      0.00      1802
         Land       0.00      0.00      0.00       402
       Loiter       0.00      0.00      0.00      9866
Obstacleavoid       0.00      0.00      0.00      2402
       Search       0.00      0.00      0.00     39505
      Takeoff       0.00      0.00      0.00       365

     accuracy                           0.09     59870
    macro avg       0.01      0.14      0.02     59870
 weighted avg       0.01      0.09      0.02     59870

[[ 5528     0     0     0     0     0     0]
 [ 1802     0     0     0     0     0     0]
 [  402     0     0     0     0     0     0]
 [ 9866     0     0     0     0     0     0]
 [ 2402     0     0     0     0     0     0]
 [39505     0     0     0     0     0     0]
 [  365     0     0     0     0     0     0]]


In [19]:
# i=0
# for (x,y) in batch_gen(X_test,Y_test,time_step,32):
#     i=i+1
#     #model.fit(x, y, epochs=1, verbose=2, shuffle=False)
#     hist = model.test_on_batch(x,y)
#     if i%100==0:
#         print(hist)
#     if i>1000:
#         break

In [20]:
# print(model.predict(x))

In [21]:
# print(y)

In [36]:
x = np.array([[.1,.2,.3],[.7,.6,.3],[.2,.2,.6],[.2,.2,.6]]) 

In [37]:
print(np.argmax(x,axis=1))

[2 0 2 2]


In [17]:
y = np.zeros((3,3))
print(y)
indxs = np.argmax(x,axis=0)
print(indxs)
y[np.arange(3),indxs]=1
print(y)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0 1 2]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [24]:
y_pred = np.eye(7)
print(encoder.inverse_transform(y_pred))

[['Hold']
 ['Hover']
 ['Land']
 ['Loiter']
 ['Obstacleavoid']
 ['Search']
 ['Takeoff']]


In [36]:
clf = tree.DecisionTreeClassifier(criterion='gini',#'entropy',#'gini', 
                                              splitter='best', 
                                              max_depth=None,#default None 
                                              min_samples_split=2, 
                                              min_samples_leaf=1, 
                                              min_weight_fraction_leaf=0.0, 
                                              max_features=None, 
                                              random_state=None, 
                                              max_leaf_nodes=None, 
                                              min_impurity_decrease=0.0, 
                                              min_impurity_split=None, 
                                              class_weight=None, 
                                              presort=False) 

In [45]:
#train decision tree
pwd = os.getcwd()
database_path = pwd[:-6] + "flightData/filtered_data/scenario5"
database_file_names = []
for root, dirs, files in os.walk(database_path):
    database_file_names = [os.path.join(root,file) for file in files]

scaler = MinMaxScaler()
train_data = None
for i,file in enumerate(database_file_names):
    data = pd.read_csv(file)
    if None is train_data:
        train_data = data
    else:
        train_data = pd.concat([train_data, data], ignore_index=True)
        
X_train = train_data.drop(columns=['label', 'time','x','y','pitch_v','roll_v','yaw_v'])
X_train = X_train.values
Y_train = train_data['label'].values
Y_train = Y_train.reshape((-1,1))

X_train = scaler.fit_transform(X_train)

clf = clf.fit(X_train, Y_train) 
# classes = ['Hold','Hover','Takeoff','Loiter','Obstacleavoid',]#pd.unique(data['label'])

In [41]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90,fontsize=16)
    plt.yticks(tick_marks, classes,fontsize=16)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    #plt.tight_layout()
    plt.ylabel('True label',fontsize=16)
    plt.xlabel('Predicted label',fontsize=16)
    return cm

In [50]:
pwd = os.getcwd()
database_path = pwd[:-6] + "flightData/filtered_data/scenario5"
database_file_names = []
for root, dirs, files in os.walk(database_path):
    database_file_names = [os.path.join(root,file) for file in files]
classes = ['Hold','Hover','Land','Loiter','Obstacleavoid','Search','Takeoff']#pd.unique(data['label'])
for i,file in enumerate(database_file_names):
    data = pd.read_csv(file)
    X_test = data.drop(columns=['label', 'time','x','y','pitch_v','roll_v','yaw_v'])
    
#     X_test = X_test.apply(lambda x: pd.to_numeric(x, errors='coerce'))
#     rows_with_nan = [index for index, row in X_test.iterrows() if row.isnull().any()]
#     X_test = X_test.drop(rows_with_nan)
    
    X_test = X_test.values
#     data = data.drop(rows_with_nan)
    Y_test = data['label'].values
    y_true = Y_test.reshape((-1,1))
    X_test = scaler.transform(X_test)
    y_pred = clf.predict(X_test) 
#     print(classification_report(y_true,y_pred))
    print(accuracy_score(y_true,y_pred))
#     conf_mat = confusion_matrix(y_true,y_pred)
#     print(conf_mat)
#     plt.figure(figsize=(6,6))
#     cm_non_normalized = plot_confusion_matrix(conf_mat, classes=classes, normalize=True,
#                       title='Normalized confusion matrix')
    #if i==0:
    #    break
# plt.show()

0.9999353211305866
0.9998998363994525
0.9999309111949772
0.9999455500803136
0.9999282588420977
